In [3]:
import glob
import numpy as np
from utils.read_ADC_bin_TDA2_separateFiles import read_adc_bin_tda2_separate_files
from utils.rangeProcCascade import rangeProc
from utils.DopplerProcCascade import dopplerProc
from utils import cfar_caso
import seaborn as sns

In [22]:
def bin2np_frame(bin_frame):  #
    neg = np.bitwise_and(bin_frame, 1 << 15) != 0
    masterArr[neg] = masterArr[neg]-(2**16)
    masterArr = masterArr[0::2] + 1j * masterArr[1::2]
    return masterArr


def get_valid_num_frames(adc_idx_file_name):
    header_info_size = 6
    with open(adc_idx_file_name, 'rb') as idx_file:
        header_info = np.fromfile(idx_file, dtype=np.uint32, count=header_info_size)
        num_idx = header_info[3]  
    
    header_info_size = 3
    with open(adc_idx_file_name, 'rb') as idx_file:
        header_info = np.fromfile(idx_file, dtype=np.uint64, count=header_info_size)
        data_file_size = header_info[2]  
    
    return num_idx, data_file_size


In [43]:
num_sample_per_chirp=256
num_chirp_per_loop=12
num_loops=64
num_rx_per_device=4
num_devices=1
chunk_size = num_sample_per_chirp*num_chirp_per_loop*num_loops*num_rx_per_device*num_devices*4
print(chunk_size)

3145728


In [24]:
filepath = r"/home/tirtha/process_mmCAS/cascade_cpu_head_1/*.bin"

In [26]:
data_files = glob.glob(filepath)
# print(data_files)
num_frames = 0 
data_file_size = 0
for file in data_files:
    if 'master_0000_idx' in file:
        num_frames, data_file_size = get_valid_num_frames(file)
        print(f"Number of Frames: {num_frames}")
        print(f"Data File Size: {data_file_size}")

Number of Frames: 99
Data File Size: 311427072


In [ ]:
angleFFTs_sum_frames = np.zeros((num_frames, 256, 256))
file_paths = {}
dtype=np.uint16
for file in data_files:
    if 'data' in file:
        if 'master' in file:
            file_paths['master'] = file
        elif 'slave1' in file:
            file_paths['slave1'] = file
        elif 'slave2' in file:
            file_paths['slave2'] = file
        elif 'slave3' in file:
            file_paths['slave3'] = file
with open(file_paths['master'], 'rb') as f_master, \
     open(file_paths['slave1'], 'rb') as f_slave1, \
     open(file_paths['slave2'], 'rb') as f_slave2, \
     open(file_paths['slave3'], 'rb') as f_slave3:
    for frame_no in range(30):
        master_arr = bin2np_frame(np.frombuffer(f_master.read(chunk_size), dtype=dtype)).reshape(num_rx_per_device, num_sample_per_chirp, num_chirp_per_loop, num_loops)
        slave1_arr = bin2np_frame(np.frombuffer(f_slave1.read(chunk_size), dtype=dtype)).reshape(num_rx_per_device, num_sample_per_chirp, num_chirp_per_loop, num_loops)
        slave2_arr = bin2np_frame(np.frombuffer(f_slave2.read(chunk_size), dtype=dtype)).reshape(num_rx_per_device, num_sample_per_chirp, num_chirp_per_loop, num_loops)
        slave3_arr = bin2np_frame(np.frombuffer(f_slave3.read(chunk_size), dtype=dtype)).reshape(num_rx_per_device, num_sample_per_chirp, num_chirp_per_loop, num_loops)
        
        print(slave3_arr)
        # bin2np_frame(master_arr)
        # adc_data = read_adc_bin_tda2_separate_files(data_files,2, num_sample_per_chirp,num_chirp_per_loop,num_loops,num_rx_per_device,num_devices)
        # rangeFFTOut = np.zeros(shape=adc_data.shape, dtype=np.complex128)
        # DopplerFFTOut = np.zeros(shape=adc_data.shape, dtype=np.complex128)
        # for i_tx in range(adc_data.shape[3]):
        #     rangeFFTOut[:,:,:,i_tx] = rangeProc(adc_data[:,:,:,i_tx])
        #     DopplerFFTOut[:,:,:,i_tx] = dopplerProc(rangeFFTOut[:,:,:,i_tx])
        # DopplerFFTOut =  np.reshape(
        #     DopplerFFTOut, 
        #     (DopplerFFTOut.shape[0], DopplerFFTOut.shape[1], DopplerFFTOut.shape[2] * DopplerFFTOut.shape[3])
        # )
        # sig_integrate = 10 * np.log10(np.sum(np.abs(DopplerFFTOut)**2, axis=2) + 1)
        break

[[[[6.5222e+04+4.8500e+02j 6.3129e+04+6.1882e+04j
    2.8500e+03+4.8590e+03j ... 6.4602e+04+6.3662e+04j
    6.3083e+04+2.7300e+02j 6.1880e+04+6.0190e+04j]
   [3.6850e+03+5.1640e+03j 6.1583e+04+6.5314e+04j
    2.9630e+03+1.8300e+02j ... 2.9850e+03+8.5900e+02j
    6.5274e+04+1.6060e+03j 6.5064e+04+6.5121e+04j]
   [2.3430e+03+3.7300e+02j 1.7140e+03+4.1930e+03j
    6.2402e+04+6.2037e+04j ... 7.5100e+02+4.0700e+02j
    1.8200e+03+1.2080e+03j 1.2320e+03+2.3540e+03j]
   ...
   [6.3258e+04+6.3818e+04j 5.7000e+01+6.4936e+04j
    6.3845e+04+6.4365e+04j ... 1.6880e+03+6.5083e+04j
    6.4071e+04+6.4132e+04j 6.4597e+04+6.4001e+04j]
   [6.5081e+04+1.3100e+03j 6.2971e+04+6.3298e+04j
    2.0440e+03+2.1190e+03j ... 3.5100e+02+6.5099e+04j
    6.5057e+04+8.5300e+02j 6.4041e+04+6.4546e+04j]
   [9.1100e+02+6.5259e+04j 1.3500e+02+1.1100e+03j
    6.3997e+04+6.4480e+04j ... 6.5197e+04+6.3801e+04j
    9.9400e+02+2.0850e+03j 6.3387e+04+6.4681e+04j]]

  [[6.4967e+04+6.4355e+04j 6.2100e+02+3.6000e+01j
    6.4062e

In [44]:
with open(r"./cascade_cpu_head_1/master_0000_data.bin","rb") as out:
    masterArr = np.frombuffer(out.read(chunk_size), dtype=np.int16)
    # masterArr = masterArr.copy()
    print(masterArr.shape)
    neg = np.bitwise_and(masterArr, 1 << 15) != 0
    masterArr[neg] = masterArr[neg]-(2**16)
    masterArr = masterArr[0::2] + 1j * masterArr[1::2]
    masterArr = np.reshape(masterArr, (num_rx_per_device, num_sample_per_chirp, num_chirp_per_loop, num_loops), order='F')
    masterArr = np.transpose(masterArr,axes=(1,3,0,2))
    print(masterArr.shape)
    print("final ", masterArr[0,0,0,:10])
    # print(masterArr.shape)

(1572864,)


OverflowError: Python integer 32768 out of bounds for int16

In [41]:
import numpy as np
import os

file_path = "./cascade_cpu_head_1/master_0000_data.bin"

# Ensure the file exists before reading
if not os.path.exists(file_path):
    raise FileNotFoundError(f"File not found: {file_path}")

# Read the entire file
with open(file_path, "rb") as out:
    data = out.read(chunk_size)

    # Ensure full buffer read
    if len(data) % 2 != 0:
        raise ValueError(f"File size ({len(data)} bytes) is not aligned to int16 (2 bytes per sample)")

    # Read as little-endian int16
    masterArr = np.frombuffer(data, dtype=np.int16)  # No need for manual conversion

# Debug print
    print("Raw data shape:", masterArr.shape)
neg = np.bitwise_and(masterArr, 1 << 15) != 0
masterArr[neg] = masterArr[neg]-(2**16)
# Convert to complex numbers (I/Q data)
masterArr = masterArr[0::2] + 1j * masterArr[1::2]

# Reshape correctly
try:
    masterArr = np.reshape(masterArr, (num_rx_per_device, num_sample_per_chirp, num_chirp_per_loop, num_loops), order='F')
    masterArr = np.transpose(masterArr, axes=(1, 3, 0, 2))
except ValueError as e:
    raise ValueError(f"Reshape failed: {e}. Check your dimensions.")

# Print debug information
print(masterArr.shape)
print("Final ", masterArr[0, 0, 0, :10])


Raw data shape: (1572864,)


OverflowError: Python integer 32768 out of bounds for int16